# Scenario 3 : square case

## library

In [33]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

required_packages = ['base', 'forecast', 'CondIndTests','RCIT'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

In [34]:
r = robjects.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')

In [35]:
RCIT = importr('RCIT')

In [36]:
pcalg = importr('pcalg')

In [37]:
def IgnorTest(cause=None, effect=None, explorer=None, covariate=None, level=0.05, dtype="continuous"): #inputs are arrays, dtypes are continuous/binary/mixed
    tmp_cause=pd.DataFrame({'A': cause})
    tmp_effect=pd.DataFrame({'Y': effect})
    tmp_explorer=pd.DataFrame({'V': explorer})
    tmp_covariate = pd.DataFrame(covariate)
    dim_covariate = tmp_covariate.shape[1]
    
    if dtype=="continuous":
        if dim_covariate==0:
            tmp_conditioning=tmp_cause
        else:
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)

        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_conditioning = ro.conversion.py2rpy(tmp_conditioning)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_effect = ro.conversion.py2rpy(tmp_effect)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_explorer = ro.conversion.py2rpy(tmp_explorer)
            
        sample_size = r.unlist(r.dim(r_from_pd_conditioning))[0]
        dim = r.unlist(r.dim(r_from_pd_conditioning))[1]
            
        r_from_pd_explorer=r.unlist(r_from_pd_explorer)    
        r_from_pd_effect=r.unlist(r_from_pd_effect)   
        r_from_pd_conditioning=r.unlist(r_from_pd_conditioning)   
        r_from_pd_conditioning=r.matrix(r_from_pd_conditioning,sample_size,dim)

        test=r.RCoT(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning,seed=2)
        pvalue=r.unlist(test)[0]
        #test=r.CondIndTest(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning, method = "KCI")
        #pvalue=r.unlist(test)[3]
        print(pvalue)
           
    elif dtype=="binary":
        if covariate==None:
            tmp_conditioning=tmp_cause
        else:
            tmp_covariate=pd.DataFrame(covariate)
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)
            
    elif dtype=="mixed":
        pass
                        
    else:
        raise ValueError("explorertype is not valid.")
        
    if(pvalue<=level):
        print("The Ignorability Assumption is not satisfied.")
    else:
        print("The Ignorability Assumption is satisfied.")

    return pvalue

### parameter setting

In [38]:
n = 4000 #changeable
c1 = 1
c2 = 1 
c3 = 1 #changeable
c4 = 1
c5 = 1
r1 = r2 = r3 = r4 = 1 #changeable
import scipy.stats as stats

### Simulation

In [39]:
#Figure 1
np.random.seed(1)

COUNT1=0 #기각 횟수
for i in range(1000):

    U = np.random.normal(0,1,n) #common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = X1**2+c1*X1+ey
    
    if i==1:
        f1_cor12=stats.pearsonr(X1, X2)[0]
    
    pvalue = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous")
    if pvalue<=0.05:
        COUNT1 = COUNT1+1

0.6537275006858353
The Ignorability Assumption is satisfied.
0.0990690988670031
The Ignorability Assumption is satisfied.
0.521282385438044
The Ignorability Assumption is satisfied.
0.32595070756014277
The Ignorability Assumption is satisfied.
0.7196315186788679
The Ignorability Assumption is satisfied.
0.9780281289920206
The Ignorability Assumption is satisfied.
0.4432840412943655
The Ignorability Assumption is satisfied.
0.961944279096876
The Ignorability Assumption is satisfied.
0.785741656028426
The Ignorability Assumption is satisfied.
0.5057684038983759
The Ignorability Assumption is satisfied.
0.909580457440635
The Ignorability Assumption is satisfied.
0.2204669938481083
The Ignorability Assumption is satisfied.
0.5845945525904936
The Ignorability Assumption is satisfied.
0.877649045456171
The Ignorability Assumption is satisfied.
0.32921120779749213
The Ignorability Assumption is satisfied.
0.3887221289581584
The Ignorability Assumption is satisfied.
0.13698967340954116
The Ign

In [40]:
#Figure 2
np.random.seed(1)

COUNT21=0 #기각 횟수
COUNT22=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = X1**2+C**2+c1*X1+c3*C+ey
    
    if i==1:
        f2_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT21 = COUNT21+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT22 = COUNT22+1 #2종 오류

0.6264817824074389
The Ignorability Assumption is satisfied.
2.155224365640862e-08
The Ignorability Assumption is not satisfied.
0.6174911468032307
The Ignorability Assumption is satisfied.
1.4621637234313312e-13
The Ignorability Assumption is not satisfied.
0.05950777247485051
The Ignorability Assumption is satisfied.
2.3377887403697173e-09
The Ignorability Assumption is not satisfied.
0.6892370471536042
The Ignorability Assumption is satisfied.
5.509037670492489e-12
The Ignorability Assumption is not satisfied.
0.7118720429274155
The Ignorability Assumption is satisfied.
1.1755085793652142e-10
The Ignorability Assumption is not satisfied.
0.22735490939908998
The Ignorability Assumption is satisfied.
4.607490400321268e-08
The Ignorability Assumption is not satisfied.
0.48748272330489817
The Ignorability Assumption is satisfied.
2.0461410343841635e-13
The Ignorability Assumption is not satisfied.
0.9066103676776501
The Ignorability Assumption is satisfied.
1.0427480785502041e-06
The Ig

In [41]:
#Figure 3
np.random.seed(1)

COUNT31=0 #기각 횟수
COUNT32=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = C**2+c3*C+ey
    
    if i==1:
        f3_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT31 = COUNT31+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT32 = COUNT32+1 #2종 오류

0.7197610643915426
The Ignorability Assumption is satisfied.
1.0769163338864018e-14
The Ignorability Assumption is not satisfied.
0.5288112062284513
The Ignorability Assumption is satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.888555193803343
The Ignorability Assumption is satisfied.
9.992007221626409e-16
The Ignorability Assumption is not satisfied.
0.2984504177163839
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.7896290061307532
The Ignorability Assumption is satisfied.
1.5543122344752192e-15
The Ignorability Assumption is not satisfied.
0.4726102106421046
The Ignorability Assumption is satisfied.
1.966913298900863e-10
The Ignorability Assumption is not satisfied.
0.9408565180377845
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8510968201373774
The Ignorability Assumption is satisfied.
2.733924198139448e-12
The Ignorability Assumption is

In [42]:
#Figure 4 
np.random.seed(1)

COUNT41=0 #기각 횟수
COUNT42=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = X1**2+X2**2+C**2+c1*X1+c4*X2+c3*C+ey
    
    if i==1:
        f4_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT41 = COUNT41+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT42 = COUNT42+1 #2종 오류

0.8187927295985888
The Ignorability Assumption is satisfied.
0.019554510741914832
The Ignorability Assumption is not satisfied.
0.4436347746264804
The Ignorability Assumption is satisfied.
0.00032649904705694954
The Ignorability Assumption is not satisfied.
0.8339267274139005
The Ignorability Assumption is satisfied.
9.210032143835978e-05
The Ignorability Assumption is not satisfied.
0.6168645713738417
The Ignorability Assumption is satisfied.
5.39484536310475e-06
The Ignorability Assumption is not satisfied.
0.24593958470263566
The Ignorability Assumption is satisfied.
0.000150932783183233
The Ignorability Assumption is not satisfied.
0.1466032668288071
The Ignorability Assumption is satisfied.
9.524497114332675e-07
The Ignorability Assumption is not satisfied.
0.3123190281554584
The Ignorability Assumption is satisfied.
0.0034673977658662425
The Ignorability Assumption is not satisfied.
0.8479042187917654
The Ignorability Assumption is satisfied.
8.940832518788966e-10
The Ignorabilit

In [43]:
#Figure 5
np.random.seed(1)

COUNT51=0 #기각 횟수
COUNT52=0
COUNT53=0
COUNT54=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = X1**2+C**2+c1*X1+c3*C+ey
    
    if i==1:
        f5_cor12=r1*r2/np.sqrt(r1**2+c2**2+1)/np.sqrt(r2**2+1)
        f5_cor13=r1*r3/np.sqrt(r1**2+c2**2+1)/np.sqrt(r3**2+1)
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT51 = COUNT51+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT52 = COUNT52+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT53 = COUNT53+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT54 = COUNT54+1 #2종 오류

0.09031275971164132
The Ignorability Assumption is satisfied.
3.8290376854765995e-06
The Ignorability Assumption is not satisfied.
0.04717294162160601
The Ignorability Assumption is not satisfied.
4.915687062956309e-07
The Ignorability Assumption is not satisfied.
0.22587420371299816
The Ignorability Assumption is satisfied.
7.121136791354843e-09
The Ignorability Assumption is not satisfied.
0.01680350055936286
The Ignorability Assumption is not satisfied.
7.770761811798366e-11
The Ignorability Assumption is not satisfied.
0.11127850240293036
The Ignorability Assumption is satisfied.
1.2212453270876722e-15
The Ignorability Assumption is not satisfied.
0.8876145606741074
The Ignorability Assumption is satisfied.
3.7580827338956624e-11
The Ignorability Assumption is not satisfied.
0.5901784807874104
The Ignorability Assumption is satisfied.
1.266690599699416e-05
The Ignorability Assumption is not satisfied.
0.7684107401935536
The Ignorability Assumption is satisfied.
3.0125254513180266e-

In [44]:
print(f5_cor12)
print(f5_cor13)

0.408248290463863
0.408248290463863


In [45]:
COUNT51

56

In [46]:
COUNT52

1000

In [47]:
COUNT53

80

In [48]:
COUNT54

1000

In [49]:
#Figure 6
np.random.seed(1)

COUNT61=0 #기각 횟수
COUNT62=0
COUNT63=0
COUNT64=0
for i in range(1000):
    U1 = np.random.normal(0,1,n) #common cause of X1 and X2
    U2 = np.random.normal(0,1,n) #common cause of X1 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U1+c2*C+r3*U2+e1
    X2 = r2*U1+e2
    X3 = r4*U2+e3
    Y = X1**2+C**2+c1*X1+c3*C+ey
    
    if i==1:
        f6_cor12=r1*r2/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r2**2+1)
        f6_cor13=r3*r4/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r4**2+1)

    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT61 = COUNT61+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT62 = COUNT62+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT63 = COUNT63+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT64 = COUNT64+1 #2종 오류

0.39967711025819797
The Ignorability Assumption is satisfied.
4.2774688691071816e-07
The Ignorability Assumption is not satisfied.
0.5066405304194195
The Ignorability Assumption is satisfied.
0.0007080283320369807
The Ignorability Assumption is not satisfied.
0.9513170607852623
The Ignorability Assumption is satisfied.
0.0024633087454617453
The Ignorability Assumption is not satisfied.
0.7036619835724287
The Ignorability Assumption is satisfied.
4.1352179769305053e-07
The Ignorability Assumption is not satisfied.
0.5785141066434063
The Ignorability Assumption is satisfied.
4.6522537044957346e-08
The Ignorability Assumption is not satisfied.
0.21232077518081316
The Ignorability Assumption is satisfied.
3.7150724552237335e-07
The Ignorability Assumption is not satisfied.
0.9239905752408862
The Ignorability Assumption is satisfied.
5.8468133512956655e-05
The Ignorability Assumption is not satisfied.
0.6757720849633192
The Ignorability Assumption is satisfied.
0.00016577472595158405
The Ig

In [50]:
print(f6_cor12)
print(f6_cor13)

0.35355339059327373
0.35355339059327373


In [51]:
COUNT61

48

In [52]:
COUNT62

993

In [53]:
COUNT63

53

In [54]:
COUNT64

992

In [55]:
#Figure 7
np.random.seed(1)

COUNT71=0 #기각 횟수
COUNT72=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder
    eu = np.random.normal(0,1,n)
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    U = c2*C+eu #common cause of X1 and X2
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = X1**2+C**2+c1*X1+c3*C+ey
    
    if i==1:
        f7_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT71 = COUNT71+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT72 = COUNT72+1 #2종 오류

0.36063111779534585
The Ignorability Assumption is satisfied.
7.216449660063518e-14
The Ignorability Assumption is not satisfied.
0.8682827135185343
The Ignorability Assumption is satisfied.
1.2445600106048005e-13
The Ignorability Assumption is not satisfied.
0.22040955266840467
The Ignorability Assumption is satisfied.
1.1903114938149884e-08
The Ignorability Assumption is not satisfied.
0.7338346461094195
The Ignorability Assumption is satisfied.
1.7877921365538896e-12
The Ignorability Assumption is not satisfied.
0.22495144716249194
The Ignorability Assumption is satisfied.
9.712345258039434e-06
The Ignorability Assumption is not satisfied.
0.3203806806087983
The Ignorability Assumption is satisfied.
5.497713395641313e-12
The Ignorability Assumption is not satisfied.
0.43494896582445697
The Ignorability Assumption is satisfied.
2.343569782681243e-12
The Ignorability Assumption is not satisfied.
0.5093860933389219
The Ignorability Assumption is satisfied.
7.603473406447847e-12
The Ign

In [56]:
#Figure 8
np.random.seed(1)

COUNT81=0 #기각 횟수
COUNT82=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder, common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*C+e1
    X2 = r2*C+e2
    Y = X1**2+C**2+c1*X1+c3*C+ey
    
    if i==1:
        f8_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT81 = COUNT81+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT82 = COUNT82+1 #2종 오류

0.4457820057556179
The Ignorability Assumption is satisfied.
2.220446049250313e-16
The Ignorability Assumption is not satisfied.
0.6041775060067285
The Ignorability Assumption is satisfied.
1.2212453270876722e-15
The Ignorability Assumption is not satisfied.
0.1028429359307117
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7944198351047307
The Ignorability Assumption is satisfied.
5.551115123125783e-16
The Ignorability Assumption is not satisfied.
0.48645864606312605
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5214736596956263
The Ignorability Assumption is satisfied.
3.3306690738754696e-16
The Ignorability Assumption is not satisfied.
0.5648636727651024
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.947635620726589
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6072696225865458
The Ignorability As

In [57]:
#Figure 9
np.random.seed(1)

COUNT91=0 #기각 횟수
COUNT92=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    Y = X1**2+C**2+c1*X1+c3*C+ey
    
    if i==1:
        f9_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT91 = COUNT91+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT92 = COUNT92+1 #2종 오류

0.5477508762012238
The Ignorability Assumption is satisfied.
1.3393643971681968e-09
The Ignorability Assumption is not satisfied.
0.6779085234400932
The Ignorability Assumption is satisfied.
6.046009692894927e-09
The Ignorability Assumption is not satisfied.
0.04088020505430956
The Ignorability Assumption is not satisfied.
1.0255463145369959e-11
The Ignorability Assumption is not satisfied.
0.9322217343059979
The Ignorability Assumption is satisfied.
1.1810928723932079e-07
The Ignorability Assumption is not satisfied.
0.39973336873674903
The Ignorability Assumption is satisfied.
5.5008442245707556e-11
The Ignorability Assumption is not satisfied.
0.5283569772485938
The Ignorability Assumption is satisfied.
1.0421671303717517e-09
The Ignorability Assumption is not satisfied.
0.4250851978178377
The Ignorability Assumption is satisfied.
7.898235843128987e-09
The Ignorability Assumption is not satisfied.
0.9191094995567564
The Ignorability Assumption is satisfied.
4.815037257799304e-13
The

In [58]:
#Figure 10
np.random.seed(1)

COUNT101=0 #기각 횟수
COUNT102=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    X3 = r3*U+e3
    Y = X1**2+C**2+X2**2+c1*X1+c3*C+c5*X2+ey
    
    if i==1:
        f10_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT101 = COUNT101+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT102 = COUNT102+1 #2종 오류

0.2385996820939611
The Ignorability Assumption is satisfied.
3.6950522641632233e-09
The Ignorability Assumption is not satisfied.
0.6101875248812224
The Ignorability Assumption is satisfied.
5.86024195836643e-09
The Ignorability Assumption is not satisfied.
0.9450859331985146
The Ignorability Assumption is satisfied.
1.79045667181299e-12
The Ignorability Assumption is not satisfied.
0.4795151732091949
The Ignorability Assumption is satisfied.
4.409472786903734e-12
The Ignorability Assumption is not satisfied.
0.10621469410305828
The Ignorability Assumption is satisfied.
4.309168466498647e-09
The Ignorability Assumption is not satisfied.
0.6488217252037765
The Ignorability Assumption is satisfied.
2.383648833870211e-12
The Ignorability Assumption is not satisfied.
0.534057731361612
The Ignorability Assumption is satisfied.
1.3990423930465568e-08
The Ignorability Assumption is not satisfied.
0.8227803921844931
The Ignorability Assumption is satisfied.
1.0735856648125264e-13
The Ignorabil

In [59]:
table1=pd.DataFrame(np.repeat(-1,70).reshape(10,7))
table1.columns = ["Scenario1","1종오류(X2)","2종오류(X2)","1종오류(X3)","2종오류(X3)","r12","r13"]
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1


In [60]:
table1["Scenario1"]=["Figure"+str(i+1) for i in range(10)]

In [61]:
table1["1종오류(X2)"]=[COUNT1,COUNT21,COUNT31,COUNT41,COUNT51,COUNT61,COUNT71,COUNT81,COUNT91,COUNT101]
table1["2종오류(X2)"]=[-1,COUNT22,COUNT32,COUNT42,COUNT52,COUNT62,COUNT72,COUNT82,COUNT92,COUNT102]
table1["1종오류(X3)"]=[-1,-1,-1,-1,COUNT53,COUNT63,-1,-1,-1,-1]
table1["2종오류(X3)"]=[-1,-1,-1,-1,COUNT54,COUNT64,-1,-1,-1,-1]
table1["r12"]=[f1_cor12,f2_cor12,f3_cor12,-1,f5_cor12,f6_cor12,f7_cor12,f8_cor12,f9_cor12,-1]
table1["r13"]=[-1,-1,-1,f4_cor13,f5_cor13,f6_cor13,-1,-1,-1,f10_cor13]

In [62]:
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,Figure1,65,-1,-1,-1,0.499098,-1.000000
1,Figure2,70,1000,-1,-1,0.413865,-1.000000
2,Figure3,50,1000,-1,-1,0.413865,-1.000000
3,Figure4,53,980,-1,-1,-1.000000,0.396688
4,Figure5,56,1000,80,1000,0.408248,0.408248
5,Figure6,48,993,53,992,0.353553,0.353553
6,Figure7,52,1000,-1,-1,0.664412,-1.000000
7,Figure8,53,1000,-1,-1,0.499098,-1.000000
8,Figure9,69,1000,-1,-1,0.664412,-1.000000
9,Figure10,53,1000,-1,-1,-1.000000,0.396688


In [63]:
#filename="SQUARE_c"+str(c3)+"_r"+str(r1)+".csv"

In [64]:
filename="SQUARE_c"+str(c3)+"_r"+str(r1)+"_s"+str(n/1000)+".csv"
table1.to_csv("C:\\Users\\minsoo\\Desktop\\실험결과(P)\\"+filename,encoding="euc-kr",index=False)